In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sys

import sys; sys.path.append(
    "/data/jerrylee/pjt/BIGFAM.v.2.0"
    )
from src import obj1, tools
import importlib

In [16]:
cohort = "UKB" # UKB, GS

# Step 1. Load FR-reg results

In [17]:
frreg_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/po-sib/frreg"

In [18]:
def select_DOR_level_frreg(fns):
    fns = [fn for fn in fns if fn.split(".")[1] in ["PC", "SB"]]
    return fns

In [19]:
fns = os.listdir(frreg_path)
fns = select_DOR_level_frreg(fns)
len(fns)

212

# Step 2. Slope test


In [20]:
output_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/po-sib/slope-test"

if os.path.exists(output_path):
    print(output_path)


/data/jerrylee/pjt/BIGFAM.v.2.0/data/UKB/po-sib/slope-test


In [21]:
for ii, fn in enumerate(tqdm(fns)):
    pheno_name = fn.split(".")[0]
    subgroup = fn.split(".")[1]
    
    # print(pheno_name, subgroup)
    
    # load FR-reg
    frreg_fn = f"{frreg_path}/{fn}"
    df_frreg = pd.read_csv(frreg_fn, delim_whitespace=True)
    
    # resampling FR-reg to compute CIs
    df_lmbds = obj1.resampleFrregCoefficients(df_frreg, n_resample=100)    
    
    # slope test(FRLog-reg)
    df_frlogreg = obj1.familialRelationshipLogRegression(df_lmbds)
    sig = obj1._slopeSig(df_frlogreg["slope"])
    
    # save slope test results
    savefn_slopetest = f"{output_path}/{pheno_name}.{subgroup}"
    (tools
     .raw2long(df_frlogreg, params=["slope", "intercept"])
     .to_csv(
         f"{savefn_slopetest}.slope", 
         sep='\t', 
         index=False)
     )
    df_frlogreg.to_csv(
        f"{savefn_slopetest}.raw.slope", 
         sep='\t',
         index=False
    )

100%|██████████| 212/212 [01:48<00:00,  1.95it/s]
